# APANPS5210 - Python for Data Analysis
## Group 67 Project

# Final Code

# SETUP

`pip install` is a command that you would run in the command prompt or terminal to install a package globally on your machine. However, in a Jupyter Notebook, you are running code in a Python environment that is specific to that notebook. Therefore, running `pip install` alone in a Jupyter Notebook cell will not install the package in the correct environment.

On the other hand, `!pip install` is a Jupyter Notebook magic command that runs the pip command in the current notebook environment. It ensures that the package is installed in the correct environment associated with the notebook.

Therefore, when you want to install a package in a Jupyter Notebook, you should use `!pip install`.

In [ ]:
!pip install py_stringsimjoin

In [ ]:
!pip install py_stringmatching

In [ ]:
!pip install ssj

In [ ]:
# We didn't use this 
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import py_stringsimjoin as ssj
import py_stringmatching as sm
import pandas as pd
import os, sys
from py_stringsimjoin.join.jaccard_join import jaccard_join
import ssj
from fuzzywuzzy import process 
import re

In [ ]:
# Local environment with VS Code

left_raw = pd.read_csv("/Users/rohit/Documents/Rohit - Master's Applied Analytics/Term 2 Courses/Python for Data Analysis/Group Project/left_dataset.csv")
right_raw = pd.read_csv("/Users/rohit/Documents/Rohit - Master's Applied Analytics/Term 2 Courses/Python for Data Analysis/Group Project/right_dataset.csv")

In [ ]:
# This way we keep the original raw data as a backup if we incorrectly modify it later
left = left_raw
right = right_raw

# FUNCTIONS
This should run in a separate .py file and we should call the functions like this:
```
from functions.py import missing_count, missing_drop
from functions.py import fix_zip_codes
from functions.py import lowercase
from functions.py import remove_substring, remove_special_chars
```

In [ ]:
# Counts the missing values in each column of df
def missing_count(df):
    result = df.isna().sum()
    print(result)

# Drops rows with missing values
def missing_drop(df):
    return df.dropna()


# Standardizes zip code
def fix_zip_codes(text):
    text = str(text)
    text = text.split("-")[0] if "-" in text else text
    text = str(int(float(text))) if "." in text else text
    return text

    
# Everything to lowercase
def lowercase(text):
    return text.lower()


# Remove substrings like LLC and INC
def remove_substring(text, substring_list):  # como se hacía para que default sea remove_substring(text, ['llc', 'inc']
    for substring in substring_list:
        text = text.replace(substring, '')
    return text    


# Remove special characters like , . / -
def remove_special_chars(text, regex_pattern): 
    return re.sub(regex_pattern, '', text)
# remove_special_chars(text, r'[^a-zA-Z0-9\s]+'))


# Remove numbers from the address # don't like it anymore
def remove_numbers(address):
    return re.sub(r'\b\d+(?![strndh])\b', '', address) #keep numbers followed by ['st', 'nd', 'rd', 'th'] because indicates a street name
# left['address_str'].apply(remove_numbers)


Our first appoach was the following.  Later we figured the second (final) approach was more flexible, as we can create a new column and not modify the original one

```
# FIX ZIP CODES
def fix_zip_codes(df, column_name):
    df[column_name] = df[column_name].astype(str)
    df[column_name] = df[column_name].apply(lambda x: x.split("-")[0] if "-" in x else x)
    df[column_name] = df[column_name].apply(lambda x: str(int(float(x))) if "." in x else x)

# calls like this:
fix_zip_codes(left, 'postal_code')


# LOWERCASE
def lowercase(df, column_name):
    df[column_name] = df[column_name].str.lower()

# calls like this: 
lowercase(left, 'name')


# REMOVE SUBSTR
def remove_substring(df, column_name, substring_list):
    for substring in substring_list:
        df[column_name] = df[column_name].str.replace(substring, '')

# calls like this:
remove_substring(left, 'name', ['llc', 'inc'])


# REMOVE SPECIAL CHARACTERS
def remove_special_chars(df, column_name, regex_pattern):
    df[column_name] = df[column_name].apply(lambda x: re.sub(regex_pattern, '', x))

# calls like this:
remove_special_chars(left, 'name', r'[^a-zA-Z0-9\s]+')
```

# DATA PREPARATION

To avoid matching n:n I will create row numbers (I think it's a good, but not-so-common, good practice when working with SQL)

The possible keys are:
* naz: name, address, zip_code
* na: name, address
* nz: name, zip_code

But first, we will clean the dataframes and call the functions to prepare our raw data

### Make the columns of both dataframes the same, so it's easier to work with

In [ ]:
left.columns, right.columns

In [ ]:
# Drop columns that don't "match" in the opposing dataset
left = left.drop(columns = ['categories'])
right = right.drop(columns = ['size'])

left.shape, right.shape

In [ ]:
# Rename columns
left = left.rename(columns = {'postal_code': 'zip_code'})
left = left.rename(columns = {'entity_id': 'left_id'})
right = right.rename(columns = {'business_id': 'right_id'})

left.columns, right.columns

In [ ]:
# This was important because some functions returned an error because of missing values

print('left', missing_count(left))   
print('right', missing_count(right)) # right doesnt' have missing values  

In [ ]:
right = missing_drop(right) 
right_raw.shape, right.shape  # right doesnt' have missing values (it doesn't print the result)

left = missing_drop(left)
left_raw.shape, left.shape

## Now we move on to the functions

In [ ]:
# FIX ZIP CODES

left['zip_code_str'] = left['zip_code'].apply(fix_zip_codes)
right['zip_code_str'] = right['zip_code'].apply(fix_zip_codes)

In [ ]:
# LOWERCASE

left['name_str'] = left['name'].apply(lowercase)
left['address_str'] = left['address'].apply(lowercase)
left['city_str'] = left['city'].apply(lowercase)

right['name_str'] = right['name'].apply(lowercase)
right['address_str'] = right['address'].apply(lowercase)
right['city_str'] = right['city'].apply(lowercase)

In [ ]:
# REMOVE SUBSTRING

left['name_str'] = left['name_str'].apply(lambda x: remove_substring(x, ['llc', 'inc']))
right['name_str'] = right['name_str'].apply(lambda x: remove_substring(x, ['llc', 'inc']))

In [ ]:
# REMOVE SPECIAL CHARACTERS

left['name_str'] = left['name_str'].apply(lambda x: remove_special_chars(x, r'[^a-zA-Z0-9\s]+'))
right['name_str'] = right['name_str'].apply(lambda x: remove_special_chars(x, r'[^a-zA-Z0-9\s]+'))

left['address_str'] = left['address_str'].apply(lambda x: remove_special_chars(x, r'[^a-zA-Z0-9\s]+'))
right['address_str'] = right['address_str'].apply(lambda x: remove_special_chars(x, r'[^a-zA-Z0-9\s]+'))

In [ ]:
left.head()

In [ ]:
right.head()

# DATA VISUALIZATION

In [ ]:
# Get a list of all states
states = [state for state in left_raw['state'] if isinstance(state, str)]

# Count the number of occurrences of each state
state_counts = pd.value_counts(states)

# Create lists of labels and values for the pie chart
labels, values = zip(*state_counts.items())

# Create the pie chart
plt.pie(values, labels=labels, autopct='%1.1f%%')
plt.title('Distribution of States in Left Dataset')
plt.show()

In [ ]:
# Get a list of all states
states = [state for state in right_raw['state'] if isinstance(state, str)]

# Count the number of occurrences of each state
state_counts = pd.value_counts(states)

# Create lists of labels and values for the pie chart
labels, values = zip(*state_counts.items())

# Create the pie chart
plt.pie(values, labels=labels, autopct='%1.1f%%')
plt.title('Distribution of States in Right Dataset')
plt.show()

In [ ]:
# Top 5 States with the Most Businesses in the left Dataset
state_counts = left_raw['state'].value_counts().nlargest(5)
plt.figure(figsize=(8,6))
plt.bar(state_counts.index, state_counts.values, color='purple')
plt.title('Top 5 States with the Most Businesses (Left Dataset)', fontsize=16)
plt.xlabel('State', fontsize=14)
plt.ylabel('Number of Businesses', fontsize=14)
plt.show()

In [ ]:
# Top 10 Cities with the Most Businesses in the Right Dataset
city_counts = right_raw['city'].value_counts().nlargest(10)
plt.figure(figsize=(18,6))
plt.bar(city_counts.index, city_counts.values, color='orange')
plt.title('Top 10 Cities with the Most Businesses (Right Dataset)', fontsize=16)
plt.xlabel('City', fontsize=14)
plt.ylabel('Number of Businesses', fontsize=14)
plt.show()

In [ ]:
# Top 5 Categories in the Left Dataset
# Create a list of all categories
all_categories = []
for cats in left_raw['categories']:
    if isinstance(cats, str):
        all_categories.extend(cats.split(','))

# Count the occurrence of each category
category_counts = {}
for category in all_categories:
    category_counts[category.strip()] = category_counts.get(category.strip(), 0) + 1

# Sort the categories by count and get the top 5
top_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)[:5]

# Create lists of the top 5 categories and their counts
labels = [x[0] for x in top_categories]
values = [x[1] for x in top_categories]

# Create a bar chart of the top 5 categories
plt.figure(figsize=(8, 6))
plt.bar(labels, values)
plt.title('Top 5 Categories (Left Dataset)', fontsize=16)
plt.xlabel('Category', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.show()

# MATCHING ALGORITHM - JACCARD

In [ ]:
A = left_raw
B = right_raw

print('Number of records in A: ' + str(len(A)))
print('Number of records in B: ' + str(len(B)))

In [ ]:
#A.entity_id, you are selecting the column with the label 'entity_id' from the DataFrame A.
A.entity_id

In [ ]:
# Create a new column in DataFrame B called 'new_key_attr', and assign a range of integers
# from 0 to the length of B to this column. This creates a unique identifier for each row in B
# which can be used for matching with the corresponding rows in DataFrame A during a fuzzy join operation.

B['new_key_attr'] = range(0, len(B))
B

In [ ]:
#Define a function to capitalize the first letter of each word in the name column
def capitalize_name(name):
    name = name.str.lower().str.title()
    return name

#Updating the returned result into the right dataset - name column
B.loc[:, 'name'] = capitalize_name(B.name)

In [ ]:
B.head()

In [ ]:
# create whitespace tokenizer for tokenizing 'name' attribute. The return_set flag should be set to True since
# Jaccard is a set based measure.
ws = sm.WhitespaceTokenizer(return_set=True)

In [ ]:
# Use the ssj library to perform a fuzzy join between DataFrames A and B based on the 'name' attribute. The 'entity_id'
# column from A will be matched against the 'business_id' column from B. The whitespace tokenizer object 'ws' will
# be used to tokenize the 'name' attributes for both DataFrames. The Jaccard similarity threshold is set to 0.8,
# meaning that pairs of rows with a Jaccard similarity score greater than or equal to 0.8 will be considered matches.
# The output_pairs DataFrame will include the 'name' attribute for both A and B for all matched pairs.
output_pairs_name = jaccard_join(A, B, 'entity_id', 'business_id', 'name', 'name', ws, 0.8,
                                l_out_attrs=['name'], r_out_attrs=['name'])

In [ ]:
output_pairs_name

In [ ]:
# Drop the '_id', 'l_name', and 'r_name' columns from the output_pairs DataFrame using the drop() method with the 'axis=1'
# parameter to indicate that the columns should be dropped. The 'inplace=True' parameter ensures that the DataFrame is
# modified in place rather than creating a new copy.

output_pairs_name.drop(['_id', 'l_name', 'r_name'], axis=1, inplace=True)

In [ ]:
output_pairs_name

In [ ]:
# Use the ssj library to perform a fuzzy join between DataFrames A and B based on the 'address' attribute. The 'entity_id'
# column from A will be matched against the 'business_id' column from B. The whitespace tokenizer object 'ws' will
# be used to tokenize the 'address' attributes for both DataFrames. The Jaccard similarity threshold is set to 0.8,
# meaning that pairs of rows with a Jaccard similarity score greater than or equal to 0.8 will be considered matches.
# The output_pairs_add DataFrame will include the 'address' attribute for both A and B for all matched pairs.


output_pairs_address = jaccard_join(A, B, 'entity_id', 'business_id', 'address', 'address', ws, 0.8,
                                l_out_attrs=['address'], r_out_attrs=['address'])

In [ ]:
# Remove the '_id', 'l_address', and 'r_address' columns from the output_pairs_add DataFrame since they are not needed.
# The 'axis=1' parameter specifies that the columns should be dropped, and the 'inplace=True' parameter specifies
# that the changes should be made to the DataFrame in place, without creating a new copy.
output_pairs_address.drop(['_id', 'l_address', 'r_address'], axis=1, inplace=True)

In [ ]:
output_pairs_address

### Generating CSV File combining the matched records for Address and Name

In [ ]:
# concatenate the 2 data frames
final_result_df = pd.concat([output_pairs_name, output_pairs_address])


#Total number of matched records for both address and name
print(final_result_df.count())

In [ ]:
# write the result data frame to a CSV file
final_result_df.to_csv('FinalResult_MatchingRecords.csv', index=False)